In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
def path_to_filename(path):
    return path.split('\\')[-1]

data_dir = "track"
columns = ["center", "left", "right", "steering", "throttle", "brake", "speed"]
data = pd.read_csv(os.path.join(data_dir, "driving_log.csv"), names=columns)

data["center"] = data["center"].apply(path_to_filename)
data["left"] = data["left"].apply(path_to_filename)
data["right"] = data["right"].apply(path_to_filename)
data.head()

,center,left,right,steering,throttle,brake,speed
0,center_2018_07_16_17_11_43_382.jpg,left_2018_07_16_17_11_43_382.jpg,right_2018_07_16_17_11_43_382.jpg,0.0,0.0,0.0,0.649786
1,center_2018_07_16_17_11_43_670.jpg,left_2018_07_16_17_11_43_670.jpg,right_2018_07_16_17_11_43_670.jpg,0.0,0.0,0.0,0.627942
2,center_2018_07_16_17_11_43_724.jpg,left_2018_07_16_17_11_43_724.jpg,right_2018_07_16_17_11_43_724.jpg,0.0,0.0,0.0,0.622910
3,center_2018_07_16_17_11_43_792.jpg,left_2018_07_16_17_11_43_792.jpg,right_2018_07_16_17_11_43_792.jpg,0.0,0.0,0.0,0.619162
4,center_2018_07_16_17_11_43_860.jpg,left_2018_07_16_17_11_43_860.jpg,right_2018_07_16_17_11_43_860.jpg,0.0,0.0,0.0,0.615438


In [3]:
def load_img_steering(data_dir, df):
    image_paths = []
    steerings = []

    for i in range(len(df)):
        indexed_data = df.iloc[i]
        center, left, right = indexed_data[0], indexed_data[1], indexed_data[2]
        image_paths.append(os.path.join(data_dir, center.strip()))
        steerings.append(float(indexed_data[3]))

        image_paths.append(os.path.join(data_dir, left.strip()))
        steerings.append(float(indexed_data[3]) + 0.15)

        image_paths.append(os.path.join(data_dir, right.strip()))
        steerings.append(float(indexed_data[3]) - 0.15)

    image_paths = np.asarray(image_paths)
    steerings = np.asarray(steerings)
    return image_paths, steerings

image_paths, steerings = load_img_steering(data_dir + "/IMG", data)


In [4]:
x_train, x_valid, y_train, y_valid = train_test_split(image_paths, steerings, test_size=0.2, random_state=6)

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class DrivingDataset(Dataset):
    def __init__(self, image_paths, steerings, transform=None):
        self.image_paths = image_paths
        self.steerings = steerings
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx])
        steering = self.steerings[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(steering, dtype=torch.float)

transform = transforms.Compose([
    transforms.Resize((66, 200)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = DrivingDataset(x_train, y_train, transform=transform)
valid_dataset = DrivingDataset(x_valid, y_valid, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

class NvidiaModel(nn.Module):
    def __init__(self):
        super(NvidiaModel, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 24, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.Conv2d(24, 36, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.Conv2d(36, 48, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.Conv2d(48, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(1152, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, 10),
            nn.ReLU(),
            nn.Linear(10, 1)
        )
    
    def forward(self, x):
        return self.model(x)

model = NvidiaModel().to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    
    for images, steerings in progress_bar:
        images, steerings = images.to(device), steerings.to(device)  
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs.squeeze(), steerings)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        progress_bar.set_postfix(loss=running_loss/len(train_loader))
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

model.eval()
validation_loss = 0.0
with torch.no_grad():
    for images, steerings in valid_loader:
        images, steerings = images.to(device), steerings.to(device)  
        
        outputs = model(images)
        loss = criterion(outputs.squeeze(), steerings)
        validation_loss += loss.item()

print(f'Validation Loss: {validation_loss/len(valid_loader)}')


c:\Users\kutay\miniconda3\envs\deepl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch [1/15], Loss: 0.0397


Epoch [2/15], Loss: 0.0219


Epoch [3/15], Loss: 0.0167


Epoch [4/15], Loss: 0.0144


...

Epoch [11/15], Loss: 0.0075


Epoch [12/15], Loss: 0.0071


Epoch [13/15], Loss: 0.0067


Epoch [14/15], Loss: 0.0064


Epoch [15/15], Loss: 0.0060
Validation Loss: 0.00786154502910856


In [6]:
model_path = "test_model_first.pth"
torch.save(model.state_dict(), model_path)


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from tqdm import tqdm
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class DrivingDataset(Dataset):
    def __init__(self, image_paths, steerings, transform=None):
        self.image_paths = image_paths
        self.steerings = steerings
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx])
        steering = self.steerings[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(steering, dtype=torch.float)


transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

train_dataset = DrivingDataset(x_train, y_train, transform=transform)
valid_dataset = DrivingDataset(x_valid, y_valid, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

class ModifiedResNet(nn.Module):
    def __init__(self):
        super(ModifiedResNet, self).__init__()
        self.resnet = models.resnet18(pretrained=True) 
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 1) 

    def forward(self, x):
        return self.resnet(x)

model = ModifiedResNet().to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    
    for images, steerings in progress_bar:
        images, steerings = images.to(device), steerings.to(device) 
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs.squeeze(), steerings)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        progress_bar.set_postfix(loss=running_loss/len(train_loader))
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

model.eval()
validation_loss = 0.0
with torch.no_grad():
    for images, steerings in valid_loader:
        images, steerings = images.to(device), steerings.to(device)  
        
        outputs = model(images)
        loss = criterion(outputs.squeeze(), steerings)
        validation_loss += loss.item()

print(f'Validation Loss: {validation_loss/len(valid_loader)}')
model_path = "test_model_second.pth"
torch.save(model.state_dict(), model_path)


c:\Users\kutay\miniconda3\envs\deepl\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\kutay\miniconda3\envs\deepl\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/15], Loss: 0.0537


Epoch [2/15], Loss: 0.0150


Epoch [3/15], Loss: 0.0114


Epoch [4/15], Loss: 0.0092


...

Epoch [11/15], Loss: 0.0052


Epoch [12/15], Loss: 0.0040


Epoch [13/15], Loss: 0.0042


Epoch [14/15], Loss: 0.0029


Epoch [15/15], Loss: 0.0035
Validation Loss: 0.0067386856198457905
